# Evaluación Completa - Validación y Consultas Abiertas

Este notebook combina:
1. Validación de datos con el 10% de los datos generados por train.ipynb
2. Consultas abiertas en lenguaje natural


## Parte 1: Validación de Datos


### Librerías


In [41]:
import os
import json
import pandas as pd
import random
from typing import List
import time
import csv
import math

import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
import shared_functions as custom_sharfun  #el archivo .py con funciones compartidas
import evaluation_metric as custom_metrics # es la ultima version del archivo de metricas

from peft import PeftModel
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from transformers import logging as hf_logging
hf_logging.set_verbosity_warning()


### Configuración del dispositivo y semillas


In [42]:
# Configuración del dispositivo
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
GLB_SEED = 42
torch.manual_seed(GLB_SEED)
random.seed(GLB_SEED)
np.random.seed(GLB_SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(GLB_SEED)


Using device: cpu


### Parámetros de configuración


In [43]:
# IMPORTANTE: dejar las mismas configuraciones que train.ipynb
# ------------------------------------------------------------------
# Evaluación
BATCH_SIZE_EVAL = 14 ## ajuste aqui segun tamaño GPU. puede ser que le toque bajar a 12 o 8

# Data / tokenization
MAX_LENGTH = 1500 # Obtenido al percentil 99% de training data
GEN_MAX_NEW_TOKENS = 377 # Ajustado a partir de analisis de percentiles

# Directorios y modelos
OUTPUT_DIR = "output/results/v09"
ADAPTER_DIR = os.path.join(OUTPUT_DIR, "modfinal")


### Cargando el modelo entrenado

**Nota importante:** Respetar `tokenizer.padding_side = "left"` y `model.eval()` para que entre en modo evaluación


In [44]:
print("Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR)
tokenizer.padding_side = "left" # A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Cargando modelo base en 4 bits...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",      # NF4 = mejor precisión
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_NAME = "Qwen/Qwen3-0.6B-Base"
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

print("Cargando modelo...")
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()


Cargando tokenizer...


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'output/results/v09/modfinal'. Use `repo_type` argument if needed.

### Cargar datos de validación


In [45]:
OUTPUT_DIR_VALDATA = os.path.join(OUTPUT_DIR, "datavalidation")
val_list = load_from_disk(OUTPUT_DIR_VALDATA)


FileNotFoundError: Directory output/results/v09/datavalidation not found

### Funciones adaptadas para ejecutar en lotes


In [46]:
def generate_json_raw_batch( texts: List[str], tokenizer, model, device, max_new_tokens: int, max_length: int, batch_size: int = 8):
    outputs = []
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    eos_brace_id = tokenizer.encode("}", add_special_tokens=False)[0]

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating", total=math.ceil(len(texts)/batch_size)):
        batch = texts[i:i + batch_size]
        prompts = [custom_sharfun.build_prompt(t) for t in batch]

        enc = tokenizer( prompts, return_tensors='pt', truncation=True, padding="longest", max_length=max_length).to(device)
        input_ids = enc["input_ids"]
        attention_mask = enc["attention_mask"]
        

        with torch.inference_mode():
            model.eval()
            out = model.generate( input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, do_sample=False, pad_token_id=pad_id, eos_token_id=eos_brace_id, use_cache=True )

        decoded = tokenizer.batch_decode(out, skip_special_tokens=True) # Decodificar outputs en lote

        # Recorte final
        cleaned = []
        for d in decoded:
            if "{" in d and "}" in d:
                first = d.find("{")
                last = d.rfind("}")
                d = d[first:last+1]
            cleaned.append(d)

        outputs.extend(cleaned)

    return outputs


In [47]:
# Generar texto bruto desde el modelo - EN MODO EVALUACIÓN
def generate_json_raw(text: str, max_new_tokens: int, max_length: int, tokenizer, model, device):
    prompt = custom_sharfun.build_prompt(text)
    enc = tokenizer( prompt, return_tensors='pt', truncation=True, padding="longest", max_length=max_length).to(device)
    input_ids = enc["input_ids"]
    attention_mask = enc["attention_mask"]
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id    
    eos_brace_id = tokenizer.encode("}", add_special_tokens=False)[0] # Forzar fin al cerrar JSON

    model.eval()
    with torch.no_grad():
        out = model.generate( input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, do_sample=False, pad_token_id=pad_id, eos_token_id=eos_brace_id, use_cache=True
                             , temperature=0.0 )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    # EXTRA: recortar SOLO el JSON final.
    if "{" in decoded and "}" in decoded:
        first = decoded.find("{")
        last = decoded.rfind("}")
        decoded = decoded[first : last + 1]

    return decoded


In [48]:
def extract_json_from_text(text: str):
    """
    Extrae el PRIMER JSON válido encontrado DESPUÉS del marcador '\nJSON:\n'.
    Escanea llaves para encontrar un bloque JSON bien balanceado.
    """
    marker = '{"buyer":'
    pos = text.find(marker)
    if pos != -1:
        start = text.find(marker)
    else: # pos == -1
        marker = "\nJSON:\n"
        pos = text.find(marker)
        if pos == -1:
            start = text.find("{")
        else:
            start = text.find("{", pos + len(marker)) # Buscar la primera llave '{' después del marcador
            if start == -1:
                start = text.find("{")

    if start == -1:
        return None

    brace_count = 0
    in_json = False

    for i in range(start, len(text)):
        if text[i] == "{":
            brace_count += 1
            in_json = True
        elif text[i] == "}":
            brace_count -= 1

            # Si brace_count llega a 0 => JSON completo
            if in_json and brace_count == 0:
                candidate = text[start:i+1]

                # intentar parsear
                try:
                    return json.loads(candidate)
                except json.JSONDecodeError:
                    # intento reemplazando comillas simples
                    try:
                        return json.loads(candidate.replace("'", '"'))
                    except Exception:
                        return None

    return None


### Ejecutar validación en lotes


In [49]:
start_time = time.time()

texts = [ex["natural_language"] for ex in val_list]
true_jsons = [ex["json_data"] for ex in val_list]
nat_langs = texts

pred_raw_list = generate_json_raw_batch( texts=texts, tokenizer=tokenizer, model=model, device=DEVICE, max_new_tokens=GEN_MAX_NEW_TOKENS,  max_length=MAX_LENGTH, batch_size=BATCH_SIZE_EVAL )
results = []
for nat_langs_save, raw, true_json in zip(nat_langs, pred_raw_list, true_jsons):
    pred_obj = extract_json_from_text(raw)
    if pred_obj is None:
        pred_obj = {}
    
    f1 = 0.0
    if pred_obj is None:
        f1 = 0.0
    else:
        try:
            f1 = custom_metrics.evaluate_json(true_json, json.dumps(pred_obj, ensure_ascii=False))
        except Exception:
            f1 = float(1.0 if pred_obj == true_json else 0.0)

    results.append({
        "nat_language": nat_langs_save,
        "raw_prediction": raw,
        "prediction": pred_obj,
        "true_json": true_json,
        "f1_score": f1
    })

end_time = time.time()
print( custom_sharfun.print_time_execution("Etapa validación de datos", start_time, end_time) )


NameError: name 'val_list' is not defined

### Visualización de resultados


In [50]:
 # Resumen de resultados en test: ver histrograma de F1 scores
f1_scores = [r['f1_score'] for r in results]
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(f1_scores) + 1), f1_scores, marker='o')
plt.title("F1 Scores por Ejemplo de Validación")
plt.xlabel("Ejemplo de Validación")
plt.ylabel("F1 Score")
plt.grid(True)
plt.show()


NameError: name 'results' is not defined

### Guardar resultados


In [51]:
# Guardar CSV - resultados de validación
OUTPUT_DIR_VAL = os.path.join(OUTPUT_DIR, "result_validation")
os.makedirs(OUTPUT_DIR_VAL, exist_ok=True)
csv_path = os.path.join(OUTPUT_DIR_VAL, 'validation_results.csv')
with open(csv_path, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['text','f1','raw','pred','true'], delimiter='|')
    writer.writeheader()
    for r in results:
        writer.writerow({
            'text': r['nat_language'],
            'f1': r['f1_score'],
            'raw': r['raw_prediction'],
            'pred': json.dumps(r['prediction'], ensure_ascii=False),
            'true': json.dumps(r['true_json'], ensure_ascii=False)
        })
print('CSV guardado en', csv_path)


NameError: name 'results' is not defined

In [52]:
# Histograma F1
f1_scores = [r['f1_score'] for r in results]
plt.figure()
plt.hist(f1_scores, bins=10)
plt.title('Distribución de F1')
plt.xlabel('F1')
plt.ylabel('Frecuencia')
plt.savefig(os.path.join(OUTPUT_DIR, 'f1_distribution.png'))
plt.show()
plt.close()
print('Histograma guardado en', os.path.join(OUTPUT_DIR, 'f1_distribution.png'))


NameError: name 'results' is not defined

---

## Parte 2: Open Evaluation - Consulta Abierta en Lenguaje Natural

Esta sección permite realizar consultas abiertas en lenguaje natural y obtener una respuesta JSON generada por el modelo entrenado.


### Función de Consulta Abierta


In [53]:
def generar_orden_compra(texto_natural: str, verbose: bool = True) -> dict:
    """
    Genera una orden de compra en formato JSON a partir de texto en lenguaje natural.
    
    Args:
        texto_natural: Texto en lenguaje natural describiendo la orden de compra
        verbose: Si True, imprime información adicional del proceso
    
    Returns:
        dict: JSON con la orden de compra estructurada
    """
    if verbose:
        print("\n" + "="*80)
        print(" TEXTO DE ENTRADA:")
        print("="*80)
        print(texto_natural)
        print("\n Generando JSON...\n")
    
    # Generar el JSON usando el modelo
    raw_output = generate_json_raw(
        text=texto_natural,
        max_new_tokens=GEN_MAX_NEW_TOKENS,
        max_length=MAX_LENGTH,
        tokenizer=tokenizer,
        model=model,
        device=DEVICE
    )
    
    # Extraer JSON del texto generado
    json_obj = extract_json_from_text(raw_output)
    
    if json_obj is None:
        if verbose:
            print(" No se pudo extraer un JSON válido")
            print("\n Salida cruda del modelo:")
            print(raw_output)
        return {}
    
    if verbose:
        print("="*80)
        print(" JSON GENERADO:")
        print("="*80)
        print(json.dumps(json_obj, indent=2, ensure_ascii=False))
        print("="*80)
    
    return json_obj


### Ejemplos de Uso

#### Ejemplo 1: Pedido Simple


In [54]:
texto_ejemplo_1 = """
Hola, necesito hacer un pedido urgente.

Mi nombre es María García y mi correo es maria.garcia@example.com
Mi teléfono es +34-555-123-456

Quiero comprar:
- 2 teclados inalámbricos
- 1 mouse ergonómico

Mi dirección es: Calle Mayor 123, Madrid, España, código postal 28013

Necesito envío express para antes del 15 de diciembre de 2025.

Gracias!
"""

resultado_1 = generar_orden_compra(texto_ejemplo_1)



 TEXTO DE ENTRADA:

Hola, necesito hacer un pedido urgente.

Mi nombre es María García y mi correo es maria.garcia@example.com
Mi teléfono es +34-555-123-456

Quiero comprar:
- 2 teclados inalámbricos
- 1 mouse ergonómico

Mi dirección es: Calle Mayor 123, Madrid, España, código postal 28013

Necesito envío express para antes del 15 de diciembre de 2025.

Gracias!


 Generando JSON...



NameError: name 'tokenizer' is not defined

#### Ejemplo 2: Pedido con Información Incompleta


#### Ejemplo 3: Tu Propia Consulta

**Escribe aquí tu propio texto para generar una orden de compra:**


In [55]:
#  Modifica el texto a continuación con tu propia orden de compra
mi_texto = """
Escribe aquí tu orden de compra en lenguaje natural...
"""

mi_resultado = generar_orden_compra(mi_texto)



 TEXTO DE ENTRADA:

Escribe aquí tu orden de compra en lenguaje natural...


 Generando JSON...



NameError: name 'tokenizer' is not defined

### Consulta Interactiva

Usa esta celda para probar múltiples consultas de forma rápida:


In [ ]:
# Escribe tu consulta aquí y ejecuta la celda
consulta = input("Escribe tu orden de compra en lenguaje natural:\n")
resultado = generar_orden_compra(consulta)


### Guardar Resultado

Si quieres guardar algún resultado específico:


In [ ]:
# Guardar el último resultado generado
output_file = "mi_orden_compra.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(resultado, f, indent=2, ensure_ascii=False)

print(f"Resultado guardado en: {output_file}")


In [ ]:
texto_ejemplo_2 = """
Hola, quiero ordenar 5 laptops para mi oficina.
Puedes contactarme al correo: juan.perez@company.com
Prefiero recogerlo yo mismo en la tienda.
"""

resultado_2 = generar_orden_compra(texto_ejemplo_2)
